In [1]:
# pip install qiskit==0.31.0

In [201]:
import qiskit
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer
import numpy as np
from qiskit.visualization import plot_histogram
from qiskit import BasicAer, execute
from tqdm import tqdm
import numpy as np
backend = BasicAer.get_backend("qasm_simulator")

def qft(num_qubits,is_inverse):
    qft_gate = QFT(num_qubits=num_qubits, do_swaps=False, inverse=is_inverse).to_gate()
    return qft_gate
def subroutine_add_const(num_qubits,const):
    qc = QuantumCircuit(num_qubits)
    for i in range(num_qubits):
        angle = const*np.pi/(2**i)
        qc.p(angle,i)
    return qc.to_gate()

In [124]:
qft_circuits=[]
for i in range(1,8):
    qft_gate = qft(num_qubits=i,is_inverse=False)
    for j in range(8-i):
        qr = QuantumRegister(7)
        cr = ClassicalRegister(7)
        qc = QuantumCircuit(qr,cr)
        qc.append(qft_gate, qr[j:j+i])
        qc.measure(qr,cr)
        qft_circuits.append(qc)
# qft_circuits[0].draw(output='mpl')

In [125]:
addition_circuits = []
with tqdm(total=10795) as pbar:
    for bits in range(1,8):
        max_num = 2**bits
        for i in range(max_num):
            for j in range(1,max_num):
                if i+j < max_num:
                    qr = QuantumRegister(7)
                    cr = ClassicalRegister(7)
                    qc = QuantumCircuit(qr,cr)
                    qft_gate = qft(num_qubits=bits,is_inverse=False)
                    qc.append(qft_gate, qr[:bits])

                    qc.append(subroutine_add_const(bits,i), qr[:bits])
                    qc.append(subroutine_add_const(bits,j), qr[:bits])

                    inverse_qft_gate = qft(num_qubits=bits,is_inverse=True)
                    qc.append(inverse_qft_gate, qr[:bits])
                    qc.measure(qr,cr)

                    # counts = execute(qc,backend,shots=100).result().get_counts()
                    # bin_val = int(list(counts.keys())[0],2)
                    pbar.update(1)
                    addition_circuits.append(qc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 10795/10795 [00:54<00:00, 199.90it/s]


In [373]:
def ghz_recursive(qc, quantum_register, low, mid, high):
    if low<mid and mid<high and low>=0 and mid>=0 and high>=0:
        # print(low, (low+mid)//2, mid, (mid+high+1)//2) 
        if low< (low+mid)//2:
            qc.cnot(quantum_register[low], quantum_register[(low+mid)//2])
        if mid < (mid+high+1)//2:
            qc.cnot(quantum_register[mid], quantum_register[(mid+high+1)//2])
        ghz_recursive(qc, quantum_register, low, (low+mid)//2, mid-1)
        ghz_recursive(qc, quantum_register, mid, (mid+high+1)//2, high)
def generate_ghz_logn(n):
    quantum_register = QuantumRegister(n)
    qc = QuantumCircuit(quantum_register)

    ####### your code goes here #######
    qc.h(quantum_register[0])
    qc.cnot(quantum_register[0], quantum_register[n//2])
    ghz_recursive(qc, quantum_register, 0, n//2, n-1)

    return qc.to_gate()


ghz_rec_circuits=[]
for i in range(2,8):
    ghz_circuit = generate_ghz_logn(i)
    for j in range(8-i):
        qr = QuantumRegister(7)
        cr = ClassicalRegister(7)
        qc = QuantumCircuit(qr,cr)
        qc.append(ghz_circuit, qr[j:j+i])
        qc.measure(qr,cr)
        ghz_rec_circuits.append(qc)
# qft_circuits[0].draw(output='mpl')

In [374]:
def generate_ghz(n):
    quantum_register = QuantumRegister(n)
    qc = QuantumCircuit(quantum_register)

    ####### your code goes here #######
    qc.h(quantum_register[0])
    for i in range(1,n):
        qc.cnot(quantum_register[0], quantum_register[i])
    return qc.to_gate()


ghz_circuits=[]
for i in range(2,8):
    ghz_circuit = generate_ghz(i)
    for j in range(8-i):
        qr = QuantumRegister(7)
        cr = ClassicalRegister(7)
        qc = QuantumCircuit(qr,cr)
        qc.append(ghz_circuit, qr[j:j+i])
        qc.measure(qr,cr)
        ghz_circuits.append(qc)

In [375]:
def oracle_func(qc,database,oracle,num_to_find):
    bin_to_find = (bin(num_to_find)[2:]).rjust(len(database),'0')
    
    bin_to_find = bin_to_find[::-1]
    for i in range(len(bin_to_find)):
        if bin_to_find[i] == '0':
            qc.x(database[i])
    qc.mct(database[:], oracle[0]) 
    for i in range(len(bin_to_find)):
        if bin_to_find[i] == '0':
            qc.x(database[i])
def grover_circuit(num_qubits, num_iteration, num_to_find):
    database = QuantumRegister(num_qubits-1)
    oracle = QuantumRegister(1)
    qc = QuantumCircuit(database,oracle)
    
    # initialization
    qc.h(database[:])
    qc.x(oracle[0])
    qc.h(oracle[0])
    
    # iters
    for j in range(num_iteration):
        # oracle
        oracle_func(qc,database,oracle,num_to_find)

        # diffusion
        qc.h(database[:])
        qc.x(database[:])
        qc.h(database[-1])
        qc.mct(database[0:-1], database[-1])
        qc.h(database[-1])
        qc.x(database[:])
        qc.h(database[:])
    qc.h(oracle[0])
    qc.x(oracle[0])
    return qc.to_gate()

In [376]:
grover_circuits=[]
with tqdm(total=868) as pbar:
    for bits in range(3,8):
        max_num = 2**(bits-1)
        iterations = int(np.floor((np.pi/4)*np.sqrt(max_num)))
        for itrs in range(1,iterations+1):
            for num_to_find in range(max_num):
                grover_gate = grover_circuit(num_qubits=bits, num_iteration=itrs, num_to_find=num_to_find)
                for j in range(8-bits):
                    qr = QuantumRegister(7)
                    cr = ClassicalRegister(7)
                    qc = QuantumCircuit(qr,cr)
                    qc.append(grover_gate, qr[j:j+bits])
                    qc.measure(qr,cr)
                    grover_circuits.append(qc)
                    pbar.update(1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 868/868 [00:12<00:00, 71.38it/s]


In [377]:
len(grover_circuits)

868

In [378]:
from qiskit.circuit.library import PhaseEstimation

In [380]:
def get_unitary(bits_for_unitary,phase):
    qc = QuantumCircuit(bits_for_unitary)
    qc.p(phase,0)
    return qc
def get_qpe(num_qubits, phase, bits_for_unitary):
    U = get_unitary(bits_for_unitary,phase)
    iqft = QFT(num_qubits=num_qubits, do_swaps=False, inverse=True)
    QPE = PhaseEstimation(num_evaluation_qubits=num_qubits, unitary=U, iqft=iqft)
    return QPE

In [382]:
def get_qpe_circuits_for_phase(phase):
    circ_list = []
    for eval_bits in range(3,8):
        for unitary_bits in range(1, 7-eval_bits+1):
            circ_size = eval_bits+unitary_bits
            qpe = get_qpe(eval_bits, phase, unitary_bits)
            for j in range(circ_size,8):
                qr = QuantumRegister(7)
                cr = ClassicalRegister(eval_bits)
                qc = QuantumCircuit(qr,cr)
                for eig_bits in range(j-1,j-1-unitary_bits,-1):
                    qc.x(qr[eig_bits])
                qc.append(qpe, qr[j-circ_size:j])
                qc.measure(qr[:eval_bits],cr)
                circ_list.append(qc)
    return circ_list
phases = [(np.random.randint(1,64))/64 for _ in range(20)]
qpe_circuits = []
for p in phases:
    qpe_circuits= qpe_circuits+get_qpe_circuits_for_phase(p)

In [383]:
test_dataset = qft_circuits + addition_circuits + ghz_rec_circuits + ghz_circuits + grover_circuits + qpe_circuits

In [392]:
depths = []
for i in tqdm(range(len(test_dataset))):
    tqc = test_dataset[i].decompose()
    if tqc.depth() not in depths:
        depths.append(tqc.depth())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 12133/12133 [01:02<00:00, 195.32it/s]


In [393]:
depths

[2, 5, 3, 4, 6, 7, 8, 12, 11, 22, 19, 32, 27, 42, 35, 52, 43, 62, 51]

In [430]:
from mlp_graph_feature import get_graph_features,clear_dataset

In [434]:
num_qubits = 7
last_num_qubits = len(df.columns)-num_qubits


iteration = [1]
for qc in test_dataset[:1]:
    data = datetime.datetime.today() - datetime.timedelta(days=random.randint(1,150))
    backends = ['ibm_lagos','ibm_perth','ibm_nairobi']
    backend = backends[np.random.randint(0,3)]
    l = add_line(qc,backend, optimization_level=3, refresh=True, show= False, datatime=data)
    
    d={}
    for i in range(len(l[0])):
        d[str(l[0][i])] = l[1][i]
    df = pd.DataFrame(d, index=[0])
    df = clear_dataset(df, 7)
    x = get_graph_features(df)
    print(x.shape)
    labels = df.iloc[:, last_num_qubits:].values
    display(df)
    print(loaded_model.predict(x))
    print(labels)

2023-11-21 22:58:57.442995
Backend topology: 2023-11-21 22:58:57.442995
[[0, 1], [1, 0], [1, 2], [1, 3], [2, 1], [3, 1], [3, 5], [4, 5], [5, 3], [5, 4], [5, 6], [6, 5]]
Depth: [6, 6, 6, 6, 6, 6]
na
{0: Qubit(QuantumRegister(7, 'q'), 0), 1: Qubit(QuantumRegister(7, 'q'), 1), 2: Qubit(QuantumRegister(7, 'q'), 2), 3: Qubit(QuantumRegister(7, 'q'), 3), 4: Qubit(QuantumRegister(7, 'q'), 4), 5: Qubit(QuantumRegister(7, 'q'), 5), 6: Qubit(QuantumRegister(7, 'q'), 6)}


ibmqfactory.load_account:WARNING:2023-11-26 22:58:59,643: Credentials are already in use. The existing account in the session will be replaced.


Connections: ['01', '10', '12', '13', '21', '31', '35', '45', '53', '54', '56', '65']


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 118.82it/s]

(1, 378)


,last_update_date,backend_name,N_qubtis,N_cx,cx_01,cx_02,cx_03,cx_04,cx_05,cx_06,...,T1_6,T2_6,readout_error_6,0,1,2,3,4,5,6
0,2023-11-21 22:32:45-08:00,ibm_perth,7,0,0,0,0,0,0,0,...,134.790035,162.022568,0.0135,0,1,2,3,4,5,6


ValueError: in user code:

    /home/ritu/anaconda3/envs/dnn_for_qubit_mapping/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1147 predict_function  *
        outputs = self.distribute_strategy.run(
    /home/ritu/anaconda3/envs/dnn_for_qubit_mapping/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/ritu/anaconda3/envs/dnn_for_qubit_mapping/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/ritu/anaconda3/envs/dnn_for_qubit_mapping/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/ritu/anaconda3/envs/dnn_for_qubit_mapping/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:1122 predict_step  **
      

In [399]:
import tensorflow as tf
from tensorflow import keras

In [400]:
from Dataset import *

In [405]:
loaded_model = tf.keras.models.load_model("models/mlp.keras")

2023-11-26 21:47:29.620831: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2023-11-26 21:47:30.837094: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-26 21:47:30.839927: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1561] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce GTX 1650 Ti computeCapability: 7.5
coreClock: 1.485GHz coreCount: 16 deviceMemorySize: 4.00GiB deviceMemoryBandwidth: 178.84GiB/s
2023-11-26 21:47:30.840144: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2023-11-26 21:47:30.840278: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynam

In [406]:
loaded_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 89)]              0         
_________________________________________________________________
dense (Dense)                (None, 70)                6300      
_________________________________________________________________
dense_1 (Dense)              (None, 49)                3479      
Total params: 9,779
Trainable params: 9,779
Non-trainable params: 0
_________________________________________________________________
